# 04 - Liquidity Risk Classification

## Aurora Utensils Manufacturing (AUM)

This notebook applies liquidity risk classification to historical and forecasted revenue data.

### Sections
1. Setup & Data Loading
2. Overview of Liquidity Metrics
3. Apply Risk Classification
4. Visualization
5. Observations

---

## 1. Setup & Data Loading

In [ ]:
# Standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os
import warnings
warnings.filterwarnings('ignore')

# Project imports
import sys
sys.path.insert(0, '../src')
from data.load_data import load_config
from data.make_features import build_liquidity_base_table
from models.liquidity import classify_liquidity, get_risk_summary, LiquidityRiskClassifier

# Plot settings
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 11
sns.set_palette('husl')

# Ensure output directories exist
os.makedirs('../data/interim', exist_ok=True)
os.makedirs('../reports/figures', exist_ok=True)

print("✓ Setup complete")
print(f"  Analysis date: {datetime.now().strftime('%Y-%m-%d %H:%M')}")

In [ ]:
# Load configuration
config = load_config()

# Display finance assumptions
print("Finance Assumptions:")
for k, v in config['finance_assumptions'].items():
    print(f"  {k}: {v}")

print("\nLiquidity Thresholds:")
print(f"  safe_min_margin: {config['liquidity_thresholds'].get('safe_min_margin', 0.15)}")
print(f"  at_risk_min_margin: {config['liquidity_thresholds'].get('at_risk_min_margin', 0.05)}")

In [ ]:
# Load historical monthly sales
df_monthly = pd.read_csv('../data/interim/monthly_sales.csv')
df_monthly['year_month'] = pd.to_datetime(df_monthly['year_month'])
print(f"Historical data: {len(df_monthly)} months")
print(f"Date range: {df_monthly['year_month'].min().strftime('%Y-%m')} to {df_monthly['year_month'].max().strftime('%Y-%m')}")

# Load 12-month forecast
df_forecast = pd.read_csv('../data/interim/revenue_forecast_12m.csv')
df_forecast['month'] = pd.to_datetime(df_forecast['month'])
print(f"\nForecast data: {len(df_forecast)} months")
print(f"Date range: {df_forecast['month'].min().strftime('%Y-%m')} to {df_forecast['month'].max().strftime('%Y-%m')}")

In [ ]:
# Build liquidity base table with historical + forecast
df_liquidity = build_liquidity_base_table(df_monthly, config, df_forecast)
print(f"\nLiquidity base table: {len(df_liquidity)} rows")
print(f"Historical: {(~df_liquidity['is_forecast']).sum()} months")
print(f"Forecast: {df_liquidity['is_forecast'].sum()} months")

---
## 2. Overview of Liquidity Metrics

In [ ]:
# Display first few rows
df_liquidity.head(10)

In [ ]:
# Summary statistics
print("Summary Statistics:")
print("="*60)
df_liquidity[['revenue', 'cogs', 'operating_cash_flow', 'adjusted_liquidity_score']].describe()

In [ ]:
# Key metrics
print("Key Metric Ranges:")
print("="*60)
print(f"Revenue (min/max): Rs.{df_liquidity['revenue'].min():,.0f} / Rs.{df_liquidity['revenue'].max():,.0f}")
print(f"Operating Cash Flow (min/max): Rs.{df_liquidity['operating_cash_flow'].min():,.0f} / Rs.{df_liquidity['operating_cash_flow'].max():,.0f}")
print(f"Adjusted Liquidity Score (min/max): {df_liquidity['adjusted_liquidity_score'].min():.4f} / {df_liquidity['adjusted_liquidity_score'].max():.4f}")
print(f"\nCash Conversion Cycle: {df_liquidity['ccc_days'].iloc[0]} days")
print(f"Operating Cash Margin: {df_liquidity['operating_cash_margin'].iloc[0]*100:.1f}%")

---
## 3. Apply Risk Classification

In [ ]:
# Apply classification
df_scored = classify_liquidity(df_liquidity, config)

# Show classification thresholds
clf = LiquidityRiskClassifier(config)
thresholds = clf.get_thresholds()
print("Classification Thresholds:")
print(f"  Safe: score >= {thresholds['safe_min_margin']}")
print(f"  At Risk: {thresholds['at_risk_min_margin']} <= score < {thresholds['safe_min_margin']}")
print(f"  Critical: score < {thresholds['at_risk_min_margin']}")

In [ ]:
# Display classification results
print("\nRisk Classification Results:")
print("="*60)
print(df_scored['liquidity_risk_label'].value_counts())

# Get summary
summary = get_risk_summary(df_scored)
print(f"\nTotal months analyzed: {summary['total_months']}")
print(f"\nHistorical period:")
for label, count in summary['historical_counts'].items():
    print(f"  {label}: {count}")
print(f"\nForecast period:")
for label, count in summary['forecast_counts'].items():
    print(f"  {label}: {count}")

In [ ]:
# Save classified table
df_scored.to_csv('../data/interim/liquidity_risk_table.csv', index=False)
print("✓ Saved to data/interim/liquidity_risk_table.csv")

In [ ]:
# Show sample of classified data
df_scored[['year_month', 'revenue', 'is_forecast', 'adjusted_liquidity_score', 'liquidity_risk_label']].tail(20)

---
## 4. Visualization

In [ ]:
# Color mapping for labels
color_map = {
    'Safe': '#2ecc71',
    'At Risk': '#f39c12',
    'Critical': '#e74c3c'
}

# Plot 1: Adjusted Liquidity Score over time with color by label
fig, ax = plt.subplots(figsize=(14, 6))

# Plot with colors by label
for label in ['Safe', 'At Risk', 'Critical']:
    mask = df_scored['liquidity_risk_label'] == label
    if mask.any():
        ax.scatter(df_scored.loc[mask, 'year_month'], 
                   df_scored.loc[mask, 'adjusted_liquidity_score'],
                   c=color_map[label], label=label, s=60, alpha=0.8, edgecolors='white')

# Add threshold lines
ax.axhline(y=thresholds['safe_min_margin'], color='#2ecc71', linestyle='--', linewidth=2, alpha=0.7, label=f'Safe threshold ({thresholds["safe_min_margin"]})')
ax.axhline(y=thresholds['at_risk_min_margin'], color='#e74c3c', linestyle='--', linewidth=2, alpha=0.7, label=f'Critical threshold ({thresholds["at_risk_min_margin"]})')

# Mark forecast boundary
forecast_start = df_scored[df_scored['is_forecast']]['year_month'].min()
ax.axvline(x=forecast_start, color='gray', linestyle=':', linewidth=2, alpha=0.7)
ax.text(forecast_start, ax.get_ylim()[1]*0.95, ' Forecast →', fontsize=10, color='gray')

ax.set_xlabel('Month')
ax.set_ylabel('Adjusted Liquidity Score')
ax.set_title('AUM Liquidity Risk Score Over Time')
ax.legend(loc='lower right')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../reports/figures/liquidity_score_timeline.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Plot 2: Bar chart of label counts (historical vs forecast)
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Overall counts
label_order = ['Safe', 'At Risk', 'Critical']
counts = df_scored['liquidity_risk_label'].value_counts().reindex(label_order, fill_value=0)
colors = [color_map[l] for l in label_order]

axes[0].bar(label_order, counts.values, color=colors, edgecolor='white', linewidth=2)
axes[0].set_xlabel('Risk Level')
axes[0].set_ylabel('Number of Months')
axes[0].set_title('Overall Risk Distribution')
for i, v in enumerate(counts.values):
    axes[0].text(i, v + 0.5, str(v), ha='center', fontweight='bold')

# Historical vs Forecast comparison
hist_counts = df_scored[~df_scored['is_forecast']]['liquidity_risk_label'].value_counts().reindex(label_order, fill_value=0)
fc_counts = df_scored[df_scored['is_forecast']]['liquidity_risk_label'].value_counts().reindex(label_order, fill_value=0)

x = np.arange(len(label_order))
width = 0.35

axes[1].bar(x - width/2, hist_counts.values, width, label='Historical', color='#3498db', edgecolor='white')
axes[1].bar(x + width/2, fc_counts.values, width, label='Forecast', color='#9b59b6', edgecolor='white')
axes[1].set_xlabel('Risk Level')
axes[1].set_ylabel('Number of Months')
axes[1].set_title('Historical vs Forecast Risk Distribution')
axes[1].set_xticks(x)
axes[1].set_xticklabels(label_order)
axes[1].legend()

plt.tight_layout()
plt.savefig('../reports/figures/liquidity_risk_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Plot 3: Revenue vs Adjusted Liquidity Score
fig, ax = plt.subplots(figsize=(10, 6))

for label in ['Safe', 'At Risk', 'Critical']:
    mask = df_scored['liquidity_risk_label'] == label
    if mask.any():
        ax.scatter(df_scored.loc[mask, 'revenue'] / 1e6, 
                   df_scored.loc[mask, 'adjusted_liquidity_score'],
                   c=color_map[label], label=label, s=80, alpha=0.7, edgecolors='white')

ax.axhline(y=thresholds['safe_min_margin'], color='#2ecc71', linestyle='--', linewidth=1.5, alpha=0.7)
ax.axhline(y=thresholds['at_risk_min_margin'], color='#e74c3c', linestyle='--', linewidth=1.5, alpha=0.7)

ax.set_xlabel('Monthly Revenue (Rs. Millions)')
ax.set_ylabel('Adjusted Liquidity Score')
ax.set_title('Revenue vs Liquidity Score')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../reports/figures/revenue_vs_liquidity.png', dpi=150, bbox_inches='tight')
plt.show()

---
## 5. Observations

### Summary of Results


In [ ]:
# Generate observations programmatically
print("="*60)
print("LIQUIDITY RISK CLASSIFICATION SUMMARY")
print("="*60)

# Overall distribution
total = len(df_scored)
for label in ['Safe', 'At Risk', 'Critical']:
    count = (df_scored['liquidity_risk_label'] == label).sum()
    pct = count / total * 100
    print(f"{label}: {count} months ({pct:.1f}%)")

print("\n" + "-"*60)
print("Historical vs Forecast Comparison:")
print("-"*60)

hist_df = df_scored[~df_scored['is_forecast']]
fc_df = df_scored[df_scored['is_forecast']]

hist_safe_pct = (hist_df['liquidity_risk_label'] == 'Safe').sum() / len(hist_df) * 100 if len(hist_df) > 0 else 0
fc_safe_pct = (fc_df['liquidity_risk_label'] == 'Safe').sum() / len(fc_df) * 100 if len(fc_df) > 0 else 0

print(f"Historical period: {len(hist_df)} months, {hist_safe_pct:.1f}% Safe")
print(f"Forecast period: {len(fc_df)} months, {fc_safe_pct:.1f}% Safe")

if fc_safe_pct < hist_safe_pct:
    print("\n⚠ Forecast months show HIGHER risk than historical months.")
elif fc_safe_pct > hist_safe_pct:
    print("\n✓ Forecast months show LOWER risk than historical months.")
else:
    print("\n→ Forecast and historical months have similar risk profiles.")

print("\n" + "-"*60)
print("Cash Conversion Cycle Analysis:")
print("-"*60)
ccc = df_scored['ccc_days'].iloc[0]
print(f"CCC: {ccc} days (DSO {df_scored['dso_days'].iloc[0]} + DIO {df_scored['inventory_days'].iloc[0]} - DPO {df_scored['dpo_days'].iloc[0]})")
print(f"Operating Cash Margin: {df_scored['operating_cash_margin'].iloc[0]*100:.1f}%")
print(f"Adjusted Score: {df_scored['adjusted_liquidity_score'].iloc[0]:.4f}")

if ccc > 60:
    print("\n⚠ CCC > 60 days indicates extended cash cycle, increasing liquidity risk.")
else:
    print("\n✓ CCC <= 60 days is within acceptable manufacturing range.")

### Key Observations

1. **Risk Distribution**: The analysis shows the distribution of Safe, At Risk, and Critical months across the historical and forecast periods.

2. **Forecast Risk Profile**: Comparing historical to forecast months reveals whether the company's liquidity position is expected to improve or deteriorate.

3. **Cash Conversion Cycle Impact**: The CCC of 65 days (DSO 45 + DIO 60 - DPO 40) creates a working capital drag that reduces the adjusted liquidity score.

4. **Consistency**: Since the working capital assumptions (DSO, DPO, inventory days) are constant, the adjusted_liquidity_score is also constant across all months. Variability comes only from revenue levels affecting operating_cash_flow.

### Recommendations

- **Reduce DSO**: Faster collection of receivables would improve the CCC and liquidity score.
- **Increase DPO**: Negotiating longer payment terms with suppliers would reduce cash cycle pressure.
- **Monitor Forecast Period**: If forecast months show elevated risk, consider building cash reserves now.